In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import json
from datetime import datetime

class TodoListApp(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title("Todo List App")
        self.geometry("500x500")  # Adjusted size

        # Create input field for adding tasks
        self.task_input = tk.Entry(self, font=("TkDefaultFont", 16), width=30, fg="gray")
        self.task_input.pack(pady=10)

        # Set placeholder for input field
        self.task_input.insert(0, "Enter your todo here...")
        self.task_input.bind("<FocusIn>", self.clear_placeholder)
        self.task_input.bind("<FocusOut>", self.restore_placeholder)

        # Button for adding tasks
        tk.Button(
            self, text="Add", command=self.add_task, font=("TkDefaultFont", 16),
            bg="#007bff", fg="white", padx=10, pady=5
        ).pack(pady=5)

        # Create a Treeview widget for displaying tasks and their date/time
        self.treeview = ttk.Treeview(self, columns=("Task", "Date Added"), show="headings", height=10)
        self.treeview.heading("Task", text="Task")
        self.treeview.heading("Date Added", text="Date Added")
        self.treeview.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        # Buttons for Done, Delete, and View Stats
        button_frame = tk.Frame(self)
        button_frame.pack(pady=10)

        tk.Button(
            button_frame, text="Done", command=self.mark_done, font=("TkDefaultFont", 16),
            bg="#28a745", fg="white", padx=10, pady=5
        ).pack(side=tk.LEFT, padx=10)

        tk.Button(
            button_frame, text="Delete", command=self.delete_task, font=("TkDefaultFont", 16),
            bg="#dc3545", fg="white", padx=10, pady=5
        ).pack(side=tk.RIGHT, padx=10)

        # View Stats button
        tk.Button(
            self, text="View Stats", command=self.view_stats, font=("TkDefaultFont", 16),
            bg="#17a2b8", fg="white", padx=10, pady=5
        ).pack(pady=10)

        self.load_tasks()

    def add_task(self):
        task = self.task_input.get()
        if task != "Enter your todo here..." and task.strip():
            # Add current date and time to the task
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            self.treeview.insert("", tk.END, values=(task, timestamp))
            self.task_input.delete(0, tk.END)
            self.save_tasks()

    def mark_done(self):
        selected_item = self.treeview.selection()
        if selected_item:
            task_text = self.treeview.item(selected_item)["values"][0]
            if not task_text.startswith("✅"):
                self.treeview.item(selected_item, values=("✅ " + task_text, self.treeview.item(selected_item)["values"][1]))

                # Update color of task to green
                self.treeview.tag_configure('done', foreground='green')
                self.treeview.item(selected_item, tags='done')
                self.save_tasks()

    def delete_task(self):
        selected_item = self.treeview.selection()
        if selected_item:
            self.treeview.delete(selected_item)
            self.save_tasks()

    def clear_placeholder(self, event):
        if self.task_input.get() == "Enter your todo here...":
            self.task_input.delete(0, tk.END)
            self.task_input.configure(fg="black")

    def restore_placeholder(self, event):
        if self.task_input.get() == "":
            self.task_input.insert(0, "Enter your todo here...")
            self.task_input.configure(fg="gray")

    def view_stats(self):
        total_count = len(self.treeview.get_children())
        done_count = 0
        for item in self.treeview.get_children():
            if "done" in self.treeview.item(item, "tags"):
                done_count += 1
        messagebox.showinfo("Task Statistics", f"Total tasks: {total_count}\nCompleted tasks: {done_count}")

    def load_tasks(self):
        try:
            with open("tasks.json", "r") as f:
                data = json.load(f)
                for task in data:
                    self.treeview.insert("", tk.END, values=(task["text"], task["timestamp"]))
        except FileNotFoundError:
            pass

    def save_tasks(self):
        data = []
        for item in self.treeview.get_children():
            task_text = self.treeview.item(item, "values")[0]
            timestamp = self.treeview.item(item, "values")[1]
            data.append({"text": task_text, "timestamp": timestamp})
        with open("tasks.json", "w") as f:
            json.dump(data, f)

if __name__ == '__main__':
    app = TodoListApp()
    app.mainloop()
